In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexNoLegend"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = InceptionV3(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 2048)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 2048)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.7min


[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.911765, total= 1.7min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.909001, total= 1.7min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.915851, total= 1.8min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.899602, total= 1.7min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.931615, total= 1.8min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.922311, total= 1.8min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.911417, total= 1.8min
[CV] C

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  4.4min


[CV] ............................ C=0.1, score=0.912745, total= 4.4min
[CV] ............................ C=0.1, score=0.932087, total= 4.4min
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.908023, total= 4.4min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.912109, total= 4.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.905567, total= 4.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.933598, total= 4.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.911968, total= 4.4min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.906863, total= 4.0min
[CV] ................... C=1000000000.0, score=0.906067, total= 4.0min
[CV] ................... C=1000000000.0, score=0.916339, total= 4.0min
[CV] ................... C=1000000000.0, score=0.906034, total= 4.1min
[CV] ................... C=1000000000.0, score=0.919722, total= 4.0min
[CV] ................... C=1000000000.0, score=0.915675, total= 4.0min
[CV] ................... C=1000000000.0, score=0.894632, total= 3.9min
[CV] .................. C=10000000000.0, score=0.906250, total= 3.5min
[CV] ................... C=1000000000.0, score=0.907371, total= 4.0min
[CV] .................. C=10000000000.0, score=0.906067, total= 3.5min
[CV] .................. C=10000000000.0, score=0.909804, total= 3.4min
[CV] .................. C=10000000000.0, score=0.888780, total= 3.3min
[CV] .................. C=10000000000.0, score=0.916339, total= 3.4min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 22.0min remaining:   52.7s


[CV] .................. C=10000000000.0, score=0.905045, total= 3.3min
[CV] .................. C=10000000000.0, score=0.920714, total= 3.3min
[CV] .................. C=10000000000.0, score=0.915675, total= 3.2min
[CV] .................. C=10000000000.0, score=0.908367, total= 3.2min
[CV] .................. C=10000000000.0, score=0.893638, total= 3.2min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 22.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([104.46195185, 266.37960067, 252.711199  , 249.67389128,
        247.74359944, 251.18526094, 248.9476599 , 252.18772388,
        247.53706522, 250.13316119, 249.50266976, 240.50754287,
        199.79291563]),
 'mean_score_time': array([0.04955146, 0.04139485, 0.04000981, 0.03570826, 0.03421566,
        0.03274062, 0.03326046, 0.03387523, 0.03095479, 0.03343723,
        0.03119338, 0.02206039, 0.01627636]),
 'mean_test_score': array([0.91525257, 0.91643646, 0.90933307, 0.90726125, 0.90647198,
        0.9066693 , 0.90686661, 0.90706393, 0.9085438 , 0.90657064,
        0.9066693 , 0.90696527, 0.90706393]),
 'mean_train_score': array([0.97490782, 0.99939715, 0.99979173, 0.99980269, 0.99966027,
        0.99978078, 0.99974788, 0.9997698 , 0.99978078, 0.99976978,
        0.99979173, 0.99978078, 0.99978078]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.91525, std: 0.00949, params: {'C': 0.01},
 mean: 0.91644, std: 0.00954, params: {'C': 0.1},
 mean: 0.90933, std: 0.00977, params: {'C': 1.0},
 mean: 0.90726, std: 0.00916, params: {'C': 10.0},
 mean: 0.90647, std: 0.00904, params: {'C': 100.0},
 mean: 0.90667, std: 0.00891, params: {'C': 1000.0},
 mean: 0.90687, std: 0.00935, params: {'C': 10000.0},
 mean: 0.90706, std: 0.00929, params: {'C': 100000.0},
 mean: 0.90854, std: 0.00822, params: {'C': 1000000.0},
 mean: 0.90657, std: 0.00906, params: {'C': 10000000.0},
 mean: 0.90667, std: 0.00920, params: {'C': 100000000.0},
 mean: 0.90697, std: 0.00863, params: {'C': 1000000000.0},
 mean: 0.90706, std: 0.00936, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.9164
